In [1]:
import numpy as np
import os
import pandas as pd
import pymssql
import time
import pickle
import gc

In [3]:
save_path = 'SAVE PATH'
conn = 'CONNECTION TO SQL SERVER'

In [4]:
df_pop = pd.read_csv(save_path+'population.csv')

In [5]:
# check that all patients are in psychosis_cohort
pc_pts = pd.io.sql.read_sql("SELECT person_id FROM results.psychosis_cohort_3yrs", conn)
print('Intersection', len(set(df_pop['person_id']).intersection(pc_pts['person_id'])), len(df_pop['person_id']))
print('Difference', len(set(df_pop['person_id']).difference(pc_pts['person_id'])))

/tmp/ipykernel_3377836/1189066159.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pc_pts = pd.io.sql.read_sql("SELECT person_id FROM results.ak4885_psychosis_cohort_3yrs", conn)


Intersection 63062 63062
Difference 0


# Medications

In [6]:
meds_query = ("SELECT pc.person_id, pc.end_date, drug_era_id, drug_concept_id, drug_era_start_date, drug_era_end_date, drug_exposure_count, gap_days "+ 
                 "FROM results.psychosis_cohort_3yrs as pc "+
                   "LEFT JOIN dbo.drug_era on drug_era.person_id = pc.person_id")                
list_chunks = []
for chunk in pd.io.sql.read_sql(meds_query, conn, chunksize=1000000):
    list_chunks.append(chunk.loc[chunk['person_id'].isin(df_pop['person_id'])])
all_meds = pd.concat(list_chunks)
all_meds.drop_duplicates(inplace=True)


/tmp/ipykernel_3377836/3326759827.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  for chunk in pd.io.sql.read_sql(meds_query, conn, chunksize=1000000):


In [8]:
all_meds = all_meds.loc[all_meds['person_id'].isin(list(df_pop['person_id']))]
all_meds = all_meds.merge(df_pop[['person_id', 'cohort_start_date', 'psychosis_diagnosis_date']], how='left', left_on = 'person_id', right_on='person_id')
all_meds = all_meds.loc[all_meds['drug_concept_id']>0]
all_meds.drop_duplicates(inplace=True)
all_meds['drug_era_id'] = all_meds['drug_era_id'].astype(int)
print(len(all_meds), len(all_meds['drug_era_id'].unique()))
# note that one drug_era_id refers to prescriptions with different start/end dates?
print(len(all_meds[['person_id', 'drug_era_id', 'drug_concept_id', 'drug_era_start_date', 'drug_era_end_date']].drop_duplicates()))

3119950 3119950
3119950


In [9]:
print(all_meds.isna().sum().sum())
print(len(all_meds))
print(len(all_meds['person_id'].unique()))
all_meds.to_csv(save_path+'temporal_medications.csv', index=False)

0
3119950
62564


# Visits

In [12]:
visits_query = ("SELECT pc.person_id, pc.end_date, visit_occurrence_id, visit_concept_id, visit_start_date, visit_end_date, visit_type_concept_id " +
                   "FROM results.psychosis_cohort_3yrs as pc "+
                   "LEFT JOIN dbo.visit_occurrence as v on v.person_id = pc.person_id")

list_chunks = []
for chunk in pd.io.sql.read_sql(visits_query, conn, chunksize=1000000):
    list_chunks.append(chunk.loc[chunk['person_id'].isin(df_pop['person_id'])])
all_visits = pd.concat(list_chunks)
all_visits.drop_duplicates(inplace=True)

/tmp/ipykernel_3377836/3905319189.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  for chunk in pd.io.sql.read_sql(visits_query, conn, chunksize=1000000):


In [13]:
all_visits = all_visits.loc[all_visits['person_id'].isin(list(df_pop['person_id']))]
all_visits = all_visits.merge(df_pop[['person_id', 'cohort_start_date', 'psychosis_diagnosis_date']], how='left', left_on = 'person_id', right_on='person_id')
all_visits = all_visits.loc[all_visits['visit_concept_id']>0]
all_visits.drop_duplicates(inplace=True)
all_visits['visit_occurrence_id'] = all_visits['visit_occurrence_id'].astype(int)
print(len(all_visits), len(all_visits['visit_occurrence_id'].unique()))

8023260 8023260


In [14]:
print(all_visits.isna().sum().sum())
print(len(all_visits))
print(len(all_visits['person_id'].unique()))
all_visits.to_csv(save_path+'temporal_visits.csv', index=False)

0
8023260
63062


In [16]:
del all_visits
gc.collect()

0

# Procedures

In [17]:
procedures_query = ("SELECT DISTINCT pc.person_id, pc.end_date, procedure_occurrence_id, procedure_date, procedure_concept_id, c.concept_name "+
                  "FROM results.psychosis_cohort_3yrs as pc "+
                  "LEFT JOIN dbo.procedure_occurrence as po on po.person_id = pc.person_id "+
                  "LEFT JOIN dbo.concept as c on c.concept_id = po.procedure_concept_id")

list_chunks = []
for chunk in pd.io.sql.read_sql(procedures_query, conn, chunksize=1000000):
    list_chunks.append(chunk.loc[chunk['person_id'].isin(df_pop['person_id'])])
all_procedures = pd.concat(list_chunks)
all_procedures.drop_duplicates(inplace=True)

/tmp/ipykernel_3377836/2806367505.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  for chunk in pd.io.sql.read_sql(procedures_query, conn, chunksize=1000000):


In [19]:
all_procedures = all_procedures.loc[all_procedures['person_id'].isin(list(df_pop['person_id']))]
all_procedures = all_procedures.loc[all_procedures['procedure_concept_id']>0]
all_procedures = all_procedures.merge(df_pop[['person_id', 'cohort_start_date', 'psychosis_diagnosis_date']], how='left', left_on = 'person_id', right_on='person_id')
all_procedures['procedure_occurrence_id'] = all_procedures['procedure_occurrence_id'].astype(int)
all_procedures.drop_duplicates(inplace=True)
print(len(all_procedures), len(all_procedures.drop_duplicates()), len(all_procedures[['person_id', 'procedure_occurrence_id', 'procedure_date', 'procedure_concept_id']].drop_duplicates()))

12152771 12152771 12152771


In [20]:
print(all_procedures.isna().sum().sum())
print(len(all_procedures))
print(len(all_procedures['person_id'].unique()))
all_procedures.to_csv(save_path+'temporal_procedures.csv', index=False)

0
12152771
63061


# Labs

In [22]:
measurements_query = ("SELECT measurement_id, pc.person_id, pc.end_date, measurement_date, measurement_concept_id, c.concept_name "+
                  "FROM results.psychosis_cohort_3yrs as pc "+
                  "LEFT JOIN dbo.measurement as m on m.person_id = pc.person_id "+
                  "LEFT JOIN dbo.concept as c on c.concept_id = m.measurement_concept_id")

list_chunks = []
for chunk in pd.io.sql.read_sql(measurements_query, conn, chunksize=1000000):
    list_chunks.append(chunk.loc[chunk['person_id'].isin(df_pop['person_id'])])
all_labs = pd.concat(list_chunks)
all_labs.drop_duplicates(inplace=True)

/tmp/ipykernel_3377836/341783702.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  for chunk in pd.io.sql.read_sql(measurements_query, conn, chunksize=1000000):


In [24]:
all_labs = all_labs.loc[all_labs['person_id'].isin(list(df_pop['person_id']))]
all_labs = all_labs.loc[all_labs['measurement_concept_id']>0]
all_labs = all_labs.merge(df_pop[['person_id', 'cohort_start_date', 'psychosis_diagnosis_date']], how='left', left_on = 'person_id', right_on='person_id')
all_labs.drop_duplicates(inplace=True)
print(len(all_labs), len(all_labs['measurement_id'].unique()))
all_labs['measurement_id'] = all_labs['measurement_id'].astype(int)
print(len(all_labs[['person_id', 'measurement_id', 'measurement_concept_id', 'measurement_date']].drop_duplicates()))

5049658 5049658
5049658


In [25]:
print(all_labs.isna().sum().sum())
print(len(all_labs))
print(len(all_labs['person_id'].unique()))
all_labs.to_csv(save_path+'temporal_labs.csv', index=False)

0
5049658
62239


In [27]:
del all_labs
gc.collect()

0

# Conditions

In [28]:
conds_query = ("SELECT condition_occurrence_id, pc.person_id, pc.end_date, condition_start_date, condition_concept_id, c.concept_name "+
                  "FROM results.ak4885_psychosis_cohort_3yrs as pc "+
                  "LEFT JOIN dbo.condition_occurrence as co on co.person_id = pc.person_id "+
                  "LEFT JOIN dbo.concept as c on c.concept_id = co.condition_concept_id "+
                  "WHERE condition_concept_id > 0")

list_chunks = []
for chunk in pd.io.sql.read_sql(conds_query, conn, chunksize=1000000):
    list_chunks.append(chunk.loc[chunk['person_id'].isin(df_pop['person_id'])])
all_conds = pd.concat(list_chunks)
all_conds.drop_duplicates(inplace=True)

/tmp/ipykernel_3377836/3283640529.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  for chunk in pd.io.sql.read_sql(conds_query, conn, chunksize=1000000):


In [29]:
all_conds = all_conds.loc[all_conds['person_id'].isin(list(df_pop['person_id']))]
all_conds = all_conds.merge(df_pop[['person_id', 'cohort_start_date', 'psychosis_diagnosis_date']], how='left', left_on = 'person_id', right_on='person_id')
all_conds.drop_duplicates(inplace=True)
all_conds['condition_occurrence_id'] = all_conds['condition_occurrence_id'].astype(int)
print(len(all_conds), len(all_conds['condition_occurrence_id'].unique()))
print(len(all_conds[['person_id', 'condition_occurrence_id', 'condition_concept_id', 'condition_start_date']].drop_duplicates()))

13924685 13924685
13924685


In [30]:
print(all_conds.isna().sum().sum())
print(len(all_conds))
print(len(all_conds['person_id'].unique()))
all_conds.to_csv(save_path+'temporal_conditions.csv', index=False)

0
13924685
63062
